In [6]:
from pathlib import Path
# PARAMETERS
NUMBER_OF_SPLITS=100
THREAD_COUNT=20

SAMPLES_DIR= Path("/scratch/enis/data/nna/labeling")
split_folder = SAMPLES_DIR / "splits"
split_length_seconds=10



In [7]:
from labeling_utils import read_file_properties,save_to_csv,cut_random_file
from multiprocessing.dummy import Pool as ThreadPool
import time

from IPython.display import display
import ipywidgets as widgets

In [4]:
file_properties,_ = read_file_properties( SAMPLES_DIR / "mp3files.txt")
mp3_files_path_list=list(file_properties.keys())


In [13]:

s=time.time()
progress = widgets.FloatProgress(value=0.0, min=0.0, max=NUMBER_OF_SPLITS)
display(progress)

def work(index):
    cut_random_file(mp3_files_path_list,length=split_length_seconds,
                    split_folder=split_folder,longest_minute=49*60,depth=0)
    progress.value+=1

pool = ThreadPool(THREAD_COUNT)
results = pool.map(work, range(NUMBER_OF_SPLITS))
pool.close()
pool.join()


FloatProgress(value=0.0)

test
test
test
test
test
test
test
test
test
test
test
Output:  Processing file '/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160715_125817.MP3' ...
 info: file matches the plugin 'mp3 (libmad)'
 info: MPEG 1 Layer 3 - 44100 Hz - Stereo - FRAME MODE - Total time: 1929m.49s
 info: starting normal split

Error: mp3splt 2.4.2 (13/05/12) - using libmp3splt 0.7.2
	Matteo Trotta <mtrotta AT users.sourceforge.net>
	Alexandru Munteanu <io_fx AT yahoo.fr>
THIS SOFTWARE COMES WITH ABSOLUTELY NO WARRANTY! USE AT YOUR OWN RISK!
 error: begin point out of file0160715_125817_2340m_08s__192..." (1 of 1)  

/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160715_125817.MP3 2340.8 2340.18
test
test
Output:  Processing file '/home/data/nna/stinchcomb/NUI_DATA/09 USGS/(3)TURBOPROPorLIGHTWEIGHT_USGS_20160610_161304.mp3' ...
 info: file matches the plugin 'mp3 (libmad)'
 info: MPEG 1 Layer 3 - 44100 Hz - Stereo - FRAME MODE - Total time: 5m.27s
 info: starting normal spl

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

range(0, 100)